<a href="https://colab.research.google.com/github/BenRyan-8/SupplyChainAnalytics_CA/blob/main/Supplier_Decision_Optimiser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok
!pip install streamlit-js-eval




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 26.5 MB/s eta 0:00:00


In [2]:
from pyngrok import ngrok
ngrok.kill()

# Replace YOUR_TOKEN with your actual ngrok token
!ngrok authtoken YOUR_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px
import streamlit.components.v1 as components
from streamlit_js_eval import streamlit_js_eval
import math

st.markdown("""
<style>
/* Remove Streamlit top padding */
.block-container {
    padding-top: 1rem;
}

/* Fix clipping of titles */
header {
    visibility: hidden;
}
</style>
""", unsafe_allow_html=True)



# ------------------ CONFIG ------------------
st.set_page_config(
    page_title="Supplier Decision Optimiser",
    layout="wide"
)

# ------------------ LOAD DATA ------------------

def load_data():
    df = pd.read_csv("/content/drive/MyDrive/FourthYear/SupplyChain/complete_supply_chain_data.csv")

    # Convert numeric fields only
    numeric_cols = [
        "Quantity_Ordered",
        "Order_Value_USD",
        "Delay_Days",
        "Days_To_Deliver",
        "Emissions_kgCO2",
        "Transport_Cost_USD"
    ]

    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

    # Derived field: Transport Cost per Unit
    df["cost_per_unit"] = df["Transport_Cost_USD"] / df["Quantity_Ordered"]
    df["damage_flag"] = df["Damaged"].apply(lambda x: 1 if x == "Y" else 0)
    df["on_time_flag"] = df["Delivery_Status"].apply(lambda x: 1 if x == "On-Time" else 0)


    return df

df = load_data()


# ------------------ Styling ------------------
card_css = """
<style>
body {
    background-color: #f4f7fb;
}
div.block-container {
    padding-top: 1rem;
}
.kpi-card {
    background: #ffffff;
    border-radius: 999px;
    padding: 18px 10px;
    text-align: center;
    box-shadow: 0 6px 16px rgba(15, 23, 42, 0.12);
}
.kpi-label {
    font-size: 0.85rem;
    color: #6b7280;
    margin-bottom: 4px;
}
.kpi-value {
    font-size: 1.2rem;
    font-weight: 700;
    color: #111827;
}
.kpi-sub {
    font-size: 0.75rem;
    color: #9ca3af;
}
.section-title {
    font-size: 1rem;
    font-weight: 600;
    color: #111827;
    margin-bottom: 0.5rem;
}
.slider-label {
    font-size: 0.8rem;
    color: #6b7280;
    margin-bottom: -6px;
}
</style>
"""
st.markdown(card_css, unsafe_allow_html=True)


# ------------------ TOP CONTROL PANEL ------------------

st.markdown("## Supplier Decision Optimiser – Hypermarket Procurement")

tab1, tab2, tab3 = st.tabs(["Supplier Rankings", "Supplier Breakdown", "Order Scenario"])

#-------Tab 1----------------

with tab1:

    st.markdown("### Supplier Rankings")

    # ------------------ CONTROL PANEL ------------------
    top_col1, top_col2 = st.columns([2, 3])

    # ===== PRODUCT SELECTION =====
    with top_col1:
        st.markdown("**Product Selection**")

        all_categories = ["All"] + sorted(df["Product_Category"].dropna().unique().tolist())
        selected_cat = st.selectbox("Product Category", all_categories)

        sub_options = (
            sorted(df["Product_Subcategory"].dropna().unique().tolist())
            if selected_cat == "All"
            else sorted(df[df["Product_Category"] == selected_cat]["Product_Subcategory"].dropna().unique().tolist())
        )
        selected_sub = st.selectbox("Product Subcategory", ["All"] + sub_options)

    # ===== KPI WEIGHTS =====
    with top_col2:
        st.markdown("**KPI Weights**")

        row1 = st.columns(2)
        row2 = st.columns(2)

        with row1[0]:
            cost_w = st.slider("Transport Cost Weight", 0, 100, 0,
                help="How important low transport cost is in the supplier ranking. Higher weight favors suppliers with lower cost per unit.")

        with row1[1]:
            speed_w = st.slider("Speed Weight", 0, 100, 0,
                help="How important fast delivery performance is. Higher weight favors suppliers with shorter average delivery time.")

        with row2[0]:
            rel_w = st.slider("Reliability Weight", 0, 100, 0,
                help="How important delivery consistency is. Reliability combines on-time performance and low damage rates.")

        with row2[1]:
            sust_w = st.slider("Sustainability Weight", 0, 100, 0,
                help="How important environmental impact is. Higher weight favors suppliers with lower CO₂ emissions per shipment.")

    # ------------------ APPLY FILTERS ------------------
    df_filtered = df.copy()

    if selected_cat != "All":
        df_filtered = df_filtered[df_filtered["Product_Category"] == selected_cat]

    if selected_sub != "All":
        df_filtered = df_filtered[df_filtered["Product_Subcategory"] == selected_sub]

    if df_filtered.empty:
        st.warning("No data available after filtering.")
        st.stop()

    # ------------------ AGGREGATE SUPPLIER KPIs ------------------
    supplier_group = df_filtered.groupby("Supplier_ID").agg(
        avg_cost_per_unit=("cost_per_unit", "mean"),
        avg_days_to_deliver=("Days_To_Deliver", "mean"),
        on_time_rate=("on_time_flag", "mean"),
        damage_rate=("damage_flag", "mean"),
        emissions=("Emissions_kgCO2", "mean")
    ).reset_index()

    # Convert percentages:
    supplier_group["on_time_rate"] *= 100
    supplier_group["damage_rate"] *= 100


    # ------------------ NORMALISE KPIs ------------------
    def normalise(series, invert=False):
        if series.max() == series.min():
            return pd.Series([50] * len(series))
        norm = (series - series.min()) / (series.max() - series.min()) * 100
        return 100 - norm if invert else norm

    supplier_group["cost_score"] = normalise(supplier_group["avg_cost_per_unit"], invert=True)
    supplier_group["speed_score"] = normalise(supplier_group["avg_days_to_deliver"], invert=True)
    supplier_group["on_time_score"] = normalise(supplier_group["on_time_rate"], invert=False)
    supplier_group["damage_score"] = normalise(supplier_group["damage_rate"], invert=True)
    supplier_group["rel_score"] = (
        0.5 * supplier_group["on_time_score"] +
        0.5 * supplier_group["damage_score"]
    )
    supplier_group["sust_score"] = normalise(supplier_group["emissions"], invert=True)

    # ------------------ WEIGHTED SCORE ------------------

    total_weight = cost_w + speed_w + rel_w + sust_w

    if total_weight == 0:
        # All weights are zero → treat all KPIs equally
        w_cost = w_speed = w_rel = w_sust = 0.25
    else:
        # Normalise weights so they sum to 1
        w_cost = cost_w / total_weight
        w_speed = speed_w / total_weight
        w_rel = rel_w / total_weight
        w_sust = sust_w / total_weight

    supplier_group["supplier_score"] = (
        w_cost * supplier_group["cost_score"] +
        w_speed * supplier_group["speed_score"] +
        w_rel  * supplier_group["rel_score"] +
        w_sust * supplier_group["sust_score"]
    )

    supplier_group = supplier_group.sort_values("supplier_score", ascending=False)

# ------------------ TABLE CLEANUP (Hide internal scores) ------------------

    display_table = supplier_group[[
        "Supplier_ID",
        "avg_cost_per_unit",
        "avg_days_to_deliver",
        "on_time_rate",
        "damage_rate",
        "emissions"
    ]].copy()

    # Add ranking column (1 = best)
    display_table.insert(0, "Rank", range(1, len(display_table) + 1))

    # Rename columns for UI
    display_table = display_table.rename(columns={
        "Supplier_ID": "Supplier",
        "avg_cost_per_unit": "Avg Transport Cost per Unit ($)",
        "avg_days_to_deliver": "Avg Days to Deliver",
        "on_time_rate": "On-Time Delivery %",
        "damage_rate": "Damage %",
        "emissions": "Avg Emissions (kg CO₂)"
    })

    # Round values
    display_table["Avg Transport Cost per Unit ($)"] = display_table["Avg Transport Cost per Unit ($)"].round(2)
    display_table["Avg Days to Deliver"] = display_table["Avg Days to Deliver"].apply(lambda x: math.ceil(x))
    display_table["On-Time Delivery %"] = display_table["On-Time Delivery %"].round(1)
    display_table["Damage %"] = display_table["Damage %"].round(1)
    display_table["Avg Emissions (kg CO₂)"] = display_table["Avg Emissions (kg CO₂)"].round(2)

    # ------------------ DISPLAY RANKING TABLE ------------------

    st.markdown("#### Ranked Suppliers (Best → Worst)")

    # -------------------------- CUSTOM CSS FOR COMPACT UI -------------------------- #
    st.markdown("""
    <style>
    /* Shorter dropdown */
    .short-dropdown > div > div {
        width: 220px !important;
    }

    /* Reduce margin above dropdown */
    .compact-label {
        margin-bottom: 20px !important;
    }

    /* KPI cards */
    .kpi-card {
        background: #ffffff;
        border-radius: 12px;
        padding: 20px;
        text-align: center;
        box-shadow: 0 2px 6px rgba(0,0,0,0.08);
        width: 100%;
    }

    .kpi-title {
        font-size: 0.75rem;
        color: #6b7280;
        font-weight: 500;
        margin-bottom: 4px;
    }

    .kpi-value {
        font-size: 2rem;
        font-weight: 500;
        color: #111827;
    }

    .kpi-row-space {
        margin-bottom: 12px;
    }

    </style>
    """, unsafe_allow_html=True)


    st.dataframe(display_table, use_container_width=True, hide_index=True)



with tab2:

    st.markdown("### Supplier KPI Breakdown")

    # -------------------------- TOP ROW LAYOUT -------------------------- #
    col_leftmid, col_radar, col_transport = st.columns([1.8, 1.4, 1.2])


    # ---------------------- LEFT + MIDDLE COLUMN ------------------------ #
    with col_leftmid:

        # Supplier selector full width
        st.markdown("<div class='compact-label'>Supplier</div>", unsafe_allow_html=True)

        suppliers_available = sorted(df_filtered["Supplier_ID"].unique().tolist())

        selected_supplier = st.selectbox(
            "",
            suppliers_available,
            key="supplier_kpi",
            help="Select a supplier to view detailed performance metrics.",
            label_visibility="collapsed",
        )

        # Apply CSS wrapper to shrink dropdown
        st.markdown("""
        <script>
        var elements = window.parent.document.querySelectorAll('.stSelectbox');
        elements.forEach(el => el.classList.add('short-dropdown'));
        </script>
        """, unsafe_allow_html=True)

        df_sup = df_filtered[df_filtered["Supplier_ID"] == selected_supplier]

        # KPI calculations
        kpi_transport_cost = df_sup["Transport_Cost_USD"].mean()
        kpi_days = df_sup["Days_To_Deliver"].mean()
        kpi_on_time = (df_sup["Delivery_Status"] == "On-Time").mean() * 100
        kpi_damage = (df_sup["Damaged"] == "Y").mean() * 100
        kpi_emissions = df_sup["Emissions_kgCO2"].mean()

        # KPI Grid Layout (Two Rows × Three Cards)
        with st.container():
            st.markdown("<div class='kpi-row-space'></div>", unsafe_allow_html=True)
            r1c1, r1c2, r1c3 = st.columns(3, gap="small")

        with st.container():
            st.markdown("<div class='kpi-row-space'></div>", unsafe_allow_html=True)
            r2c1, r2c2, r2c3 = st.columns(3, gap="small")

        def kpi_card(col, title, value):
            with col:
                st.markdown(
                    f"""
                    <div class='kpi-card'>
                        <div class='kpi-title'>{title}</div>
                        <div class='kpi-value'>{value}</div>
                    </div>
                    """,
                    unsafe_allow_html=True
                )

        # Row 1
        kpi_card(r1c1, "Avg Cost", f"${kpi_transport_cost:,.1f}")
        kpi_card(r1c2, "Avg Days", f"{kpi_days:.1f}")
        kpi_card(r1c3, "Emissions", f"{kpi_emissions:.1f} CO₂")

        # Row 2
        kpi_card(r2c1, "On-Time %", f"{kpi_on_time:.1f}%")
        kpi_card(r2c2, "Damage %", f"{kpi_damage:.1f}%")
        kpi_card(r2c3, "Orders", len(df_sup))

    # -------------------------- RADAR COLUMN ----------------------------- #
    with col_radar:
        st.markdown("#### KPI Radar Profile")

        # ---------- NORMALISED KPI SCORES ----------
        def safe_norm(val, minv, maxv, invert=False):
            if maxv == minv:
                return 50
            score = (val - minv) / (maxv - minv) * 100
            return 100 - score if invert else score

        cost_score = safe_norm(kpi_transport_cost,
                              df_filtered["Transport_Cost_USD"].min(),
                              df_filtered["Transport_Cost_USD"].max(),
                              invert=True)

        speed_score = safe_norm(kpi_days,
                                df_filtered["Days_To_Deliver"].min(),
                                df_filtered["Days_To_Deliver"].max(),
                                invert=True)

        rel_score = (0.5 * kpi_on_time) + (0.5 * (100 - kpi_damage))

        sust_score = safe_norm(kpi_emissions,
                              df_filtered["Emissions_kgCO2"].min(),
                              df_filtered["Emissions_kgCO2"].max(),
                              invert=True)

        radar_scores = [cost_score, speed_score, rel_score, sust_score]
        kpi_labels = ["Cost", "Speed", "Reliability", "Sustainability"]

        # ----------- RADAR CHART WITH FIXED LABEL CUT-OFF ----------- #
        import plotly.graph_objects as go

        fig_radar = go.Figure()

        fig_radar.add_trace(go.Scatterpolar(
            r=radar_scores + [radar_scores[0]],
            theta=kpi_labels + [kpi_labels[0]],
            fill='toself',
            line=dict(color='royalblue', width=2)
        ))

        fig_radar.update_layout(
            polar=dict(
                radialaxis=dict(visible=True, range=[0, 100]),
                angularaxis=dict(showline=True, linewidth=1),
            ),
            showlegend=False,
            height=330,
            margin=dict(t=40, b=40, l=40, r=40)
        )

        st.plotly_chart(fig_radar, use_container_width=True)




        # --------------------- NEW TOP RIGHT CHART ------------------------- #
    with col_transport:
        st.markdown("#### Orders by Transport Mode")

        # Build a clean transport-mode dataframe
        transport_df = (
            df_sup["Shipping_Mode"]
                .value_counts()
                .reset_index()
                .rename(columns={"index": "Mode", "Shipping_Mode": "Order Count"})
        )

        # Ensure the column names exist exactly as expected
        transport_df.columns = ["Mode", "Order Count"]

        # Now plot
        fig_transport = px.bar(
            transport_df,
            x="Mode",
            y="Order Count",
            color="Mode",
            text_auto=True
        )

        fig_transport.update_layout(
            height=300,
            showlegend=False,
            margin=dict(l=10, r=10, t=40, b=10)
        )

        st.plotly_chart(fig_transport, use_container_width=True)


        # ------------------------------------------------------
        #     CHARTS SECTION — THREE CHARTS SIDE BY SIDE
        # ------------------------------------------------------

    colA, colB, colC = st.columns(3, gap="large")

    # ============================== CHART 1 ============================== #
    # Orders by Subcategory
    with colA:
        st.markdown("#### Orders by Subcategory")

        orders_df = (
            df_sup.groupby("Product_Subcategory")["Order_ID"]
                  .count()
                  .reset_index()
                  .rename(columns={"Order_ID": "Order Count"})
        )

        fig_orders = px.bar(
            orders_df,
            x="Product_Subcategory",
            y="Order Count",
            color="Product_Subcategory",
            text_auto=True,
        )
        fig_orders.update_layout(
            height=300,
            showlegend=False,
            margin=dict(l=10, r=10, t=40, b=10),
        )

        st.plotly_chart(fig_orders, use_container_width=True)


    # ============================== CHART 2 ============================== #
    # Damage by Subcategory
    with colB:
        st.markdown("#### Damage % by Subcategory")

        damage_df = (
            df_sup.groupby("Product_Subcategory")["Damaged"]
                  .apply(lambda x: (x == "Y").mean() * 100)
                  .reset_index()
                  .rename(columns={"Damaged": "Damage %"})
        )

        fig_damage = px.bar(
            damage_df,
            x="Product_Subcategory",
            y="Damage %",
            text_auto=".1f",
            color="Product_Subcategory",
        )
        fig_damage.update_layout(
            height=300,
            showlegend=False,
            margin=dict(l=10, r=10, t=40, b=10),
        )

        st.plotly_chart(fig_damage, use_container_width=True)



    # ============================== CHART 3 ============================== #
    # Lead-Time Variability (Box Plot)
    with colC:
        st.markdown("#### Lead-Time Variability")

        fig_lead = px.box(
            df_sup,
            y="Days_To_Deliver",
            points="all",
            labels={"Days_To_Deliver": "Days to Deliver"},
            color_discrete_sequence=["#4e79a7"]
        )

        fig_lead.update_layout(
            height=300,
            showlegend=False,
            margin=dict(l=10, r=10, t=40, b=10),
        )

        st.plotly_chart(fig_lead, use_container_width=True)

# ==========================================================
#   THIRD ROW — DISRUPTION SUMMARY (FOUR COLUMNS)
# ==========================================================

    st.markdown("#### Disruption Summary")

    disruption_df = df_sup[df_sup["Delivery_Status"].isin(["Late", "Cancelled"])].copy()

    if disruption_df.empty:
        st.success("This supplier has no late or cancelled orders. ✔️")

    else:
        # -------- Prepare summary values -------- #
        top_cause = disruption_df["Disruption_Type"].value_counts().idxmax()
        top_cause_pct = round(
            disruption_df["Disruption_Type"].value_counts(normalize=True).loc[top_cause] * 100,
            1
        )

        disruption_rate = round((len(disruption_df) / len(df_sup)) * 100, 1)

        status_split = disruption_df["Delivery_Status"].value_counts(normalize=True).mul(100).round(1)
        late_pct = status_split.get("Late", 0)
        cancelled_pct = status_split.get("Cancelled", 0)

        top_subcat = disruption_df["Product_Subcategory"].value_counts().idxmax()

        # -------- Display in 4 cards across the width -------- #
        d1, d2, d3, d4 = st.columns(4)

        def disruption_card(col, title, value):
            with col:
                st.markdown(f"""
                <div style="
                    background-color: #ffffff;
                    padding: 14px;
                    border-radius: 12px;
                    text-align: center;
                    box-shadow: 0 2px 6px rgba(0,0,0,0.08);
                ">
                    <div style="font-size:0.85rem; color:#6b7280;">{title}</div>
                    <div style="font-size:1.25rem; font-weight:700; margin-top:4px;">{value}</div>
                </div>
                """, unsafe_allow_html=True)

        # 4 metric cards
        disruption_card(d1, "Top Cause", f"{top_cause} ({top_cause_pct}%)")
        disruption_card(d2, "Disruption Rate", f"{disruption_rate}%")
        disruption_card(d3, "Late vs Cancelled", f"{late_pct}% / {cancelled_pct}%")
        disruption_card(d4, "Most Affected Subcategory", top_subcat)

#------------------------------------------------------------------------------------------------------------
with tab3:
    st.markdown("### Order Scenario Simulator")

    # ------------------ FILTER BASED ON TAB 1 SELECTION ------------------
    df_filtered = df.copy()

    if 'selected_cat' in globals() and selected_cat != "All":
        df_filtered = df_filtered[df_filtered["Product_Category"] == selected_cat]

    if 'selected_sub' in globals() and selected_sub != "All":
        df_filtered = df_filtered[df_filtered["Product_Subcategory"] == selected_sub]

    # ------------------ TOP ROW: SUPPLIER + ORDER INPUTS ------------------
    top1, top2 = st.columns([1, 2])

    with top1:
        suppliers_available = sorted(df_filtered["Supplier_ID"].unique().tolist())
        selected_supplier_scenario = st.selectbox(
            "Supplier",
            suppliers_available,
            key="supplier_scenario",
            help="Select a supplier for scenario estimation."
        )

        df_sup = df_filtered[df_filtered["Supplier_ID"] == selected_supplier_scenario]

        if df_sup.empty:
            st.warning("No data available for this supplier within selected filters.")
            st.stop()

    with top2:

        # One single row containing both inputs
        inp1, inp2 = st.columns(2)

        with inp1:
            order_qty_scenario = st.number_input(
                "Order Quantity",
                min_value=1,
                value=500,
                step=10,
                help="Number of units you plan to order from the selected supplier."
            )

        with inp2:
            order_val_scenario = st.number_input(
                "Order Value (USD)",
                min_value=0.0,
                value=20000.0,
                step=100.0,
                help="Total product cost you will pay to the supplier for this order."
            )


    # ------------------ COMPUTE ESTIMATED OUTCOMES ------------------
    avg_cost_per_unit = df_sup["Transport_Cost_USD"].sum() / df_sup["Quantity_Ordered"].sum()
    est_transport_cost = avg_cost_per_unit * order_qty_scenario

    est_delivery_days = df_sup["Days_To_Deliver"].mean()

    est_emissions_unit = df_sup["Emissions_kgCO2"].sum() / df_sup["Quantity_Ordered"].sum()
    est_total_emissions = est_emissions_unit * order_qty_scenario

    on_time_probability = (df_sup["Delivery_Status"] == "On-Time").mean() * 100
    damage_probability = (df_sup["Damaged"] == "Y").mean() * 100

    total_landed_cost = order_val_scenario + est_transport_cost

    # ------------------ KPI OUTPUT CARDS (2 ROWS × 3 COLUMNS) ------------------
    st.markdown("### Estimated Order Outcomes")

    row1_col1, row1_col2, row1_col3 = st.columns(3)

    row1_col1.metric(
        "Estimated Transport Cost",
        f"${est_transport_cost:,.0f}",
        help="Transport cost predicted from supplier's historical cost per unit."
    )

    row1_col2.metric(
        "Estimated Delivery Time",
        f"{est_delivery_days:.1f} days",
        help="Average delivery time based on historical supplier performance."
    )

    row1_col3.metric(
        "Estimated Total Emissions",
        f"{est_total_emissions:.1f} kg CO₂",
        help="Calculated using historical emission-per-unit averages."
    )

    row2_col1, row2_col2, row2_col3 = st.columns(3)

    row2_col1.metric(
        "On-Time Delivery Probability",
        f"{on_time_probability:.1f}%",
        help="Percentage of prior orders delivered on time."
    )

    row2_col2.metric(
        "Damage Risk",
        f"{damage_probability:.1f}%",
        help="Percentage of prior orders that arrived damaged."
    )

    row2_col3.metric(
        "Total Landed Cost",
        f"${total_landed_cost:,.0f}",
        help="Order value + estimated transport cost."
    )


#------------------------------------------------------------------------------------------------------------


Writing app.py


In [4]:
# Run streamlit in the background
import subprocess
process = subprocess.Popen(["streamlit", "run", "app.py"])

# Open a public URL for the Streamlit app
public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://subsinuous-ellis-smearier.ngrok-free.dev" -> "http://localhost:8501">